### Python libraries

In [ ]:
import xarray as xr
import pandas as pd
import pyproj

from pathlib import Path

from datetime import datetime
from metloom.pointdata import SnotelPointData

### Local python helpers

In [ ]:
from dask_utils import start_cluster, client_ip_and_port

In [ ]:
%load_ext autoreload
%autoreload 2

## File path to data

In [ ]:
SNOBAL_DIR = Path('/pl/active/McGrath_Snow/isnobal')
STATION_DIR = Path('/pl/active/McGrath_Snow/AWS-data')

## Coordinate transformer

In [ ]:
wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:26913')
transformer = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True)

## SNOTEL site

### Convert SNOTEL coords to UTM

In [ ]:
latitude = 40.53284135
longitude = -105.8867646

utm_x, utm_y = transformer.transform(longitude, latitude)

print(f"UTM Coordinates: X: {utm_x}, Y: {utm_y}")

### Load data

In [ ]:
jw_snotel_point = SnotelPointData("551:CO:SNTL", "Joe Wright")

jw_snotel = jw_snotel_point.get_daily_data(
    datetime(2020, 10, 1), datetime(2024, 7, 31),
    [jw_snotel_point.ALLOWED_VARIABLES.SNOWDEPTH]
)

jw_snotel['SNOWDEPTH_M'] = jw_snotel.SNOWDEPTH * 0.0254

jw_snotel_mt = jw_snotel['SNOWDEPTH_M'].reset_index().set_index("datetime").tz_convert('US/Mountain')['SNOWDEPTH_M']

## Station data

### Coordinates

In [ ]:
burned_north_lon = -105.8468274835282
burned_north_lat = 40.54931087929571

burned_north_x, burned_north_y = transformer.transform(burned_north_lon, burned_north_lat)

burned_south_lon = -105.8690016479696
burned_south_lat = 40.55922870450481

burned_south_x, burned_south_y = transformer.transform(burned_south_lon, burned_south_lat)

burned_flat_lon = -105.8669469812113
burned_flat_lat = 40.56410945571656

burned_flat_x, burned_flat_y = transformer.transform(burned_flat_lon, burned_flat_lat)

unburned_north_lon = -105.8696044592018
unburned_north_lat = 40.56453355739959

unburned_north_x, unburned_north_y = transformer.transform(unburned_north_lon, unburned_north_lat)

unburned_flat_lon = -105.8913545858411
unburned_flat_lat = 40.53425307566753

unburned_flat_x, unburned_flat_y = transformer.transform(unburned_flat_lon, unburned_flat_lat)

### Load Data

In [ ]:
PD_PARSE_OPTS = dict(header=0, index_col=0, parse_dates=True)
bn_depth = pd.read_csv(STATION_DIR / 'B_North_daily_020525.csv', **PD_PARSE_OPTS)
bs_depth = pd.read_csv(STATION_DIR / 'B_South_daily_020525.csv', **PD_PARSE_OPTS)
bf_depth = pd.read_csv(STATION_DIR / 'B_daily_020525.csv', **PD_PARSE_OPTS)
un_depth = pd.read_csv(STATION_DIR / 'UB_North_daily_020525.csv', **PD_PARSE_OPTS)
uf_depth = pd.read_csv(STATION_DIR / 'UB_daily_020525.csv', **PD_PARSE_OPTS)

## iSnobal

### Start parallel helper workers

In [ ]:
client = start_cluster(10, 32)
client_ip_and_port(client)

### Load data

In [ ]:
wy_snow = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy202[1,2,3,4]' / 'cp/*/snow.nc').as_posix(),
    preprocess=lambda ds: ds['thickness'],
    chunks="auto",
    parallel=True,
)

In [ ]:
jw_snobal = wy_snow.sel(x=utm_x, y=utm_y, method='nearest').compute()

In [ ]:
bn_snobal = wy_snow.sel(x=burned_north_x, y=burned_north_y, method='nearest').compute()

In [ ]:
bs_snobal = wy_snow.sel(x=burned_south_x, y=burned_south_y, method='nearest').compute()

In [ ]:
bf_snobal = wy_snow.sel(x=burned_flat_x, y=burned_flat_y, method='nearest').compute()

In [ ]:
un_snobal = wy_snow.sel(x=unburned_north_x, y=unburned_north_y, method='nearest').compute()

In [ ]:
uf_snobal = wy_snow.sel(x=unburned_flat_x, y=unburned_flat_y, method='nearest').compute()

In [ ]:
client.shutdown()

## Plots

In [ ]:
import hvplot.xarray
import hvplot.pandas
import holoviews as hv

hvplot.extension('bokeh')
pd.options.plotting.backend = 'holoviews'

In [ ]:
hv.output(fig='auto', dpi=300)

HV_PLOT_OPTS = dict(
    width=1200,
    height=600,
    ylabel='Snow Depth (m)',
    xlabel='Water Year'
)

SNOTEL_LINE = dict(
    line_width=3
)

### Burned North

In [ ]:
bn_snobal.thickness.hvplot(label='iSnobal', **HV_PLOT_OPTS) * \
bn_depth['DBTCDT_Avg'].hvplot(label='Station')

### Burned South

In [ ]:
bs_snobal.thickness.hvplot(label='iSnobal', **HV_PLOT_OPTS) * \
bs_depth['DBTCDT_Avg'].hvplot(label='Station')

### Burned Flat

In [ ]:
bf_snobal.thickness.hvplot(label='iSnobal', **HV_PLOT_OPTS) * \
bf_depth['DBTCDT_Avg'].hvplot(label='Station')

### Unburned North

In [ ]:
un_snobal.thickness.hvplot(label='iSnobal', **HV_PLOT_OPTS) * \
un_depth['DBTCDT_Avg'].hvplot(label='Station')

### Unburned Flat

In [ ]:
uf_snobal.thickness.hvplot(label='iSnobal', **HV_PLOT_OPTS) * \
uf_depth['DBTCDT_Avg'].hvplot(label='Station')

### SNOTEL

In [ ]:
jw_snobal.thickness.hvplot(label='iSnobal', **HV_PLOT_OPTS) * \
jw_snotel_mt.plot(label='Station', **SNOTEL_LINE)